In [ ]:
from __future__ import print_function, division

import numpy as np
import pandas as pd

import astropy.units as u
import astropy.coordinates as apycoord
from astropy.visualization import scale_image
from astropy.nddata import block_replicate
from astropy.table import Table, Column, MaskedColumn
from astropy.time import Time

import glowing_waffles.differential_photometry as df

from glowing_waffles.io import parse_aij_table

from ccdproc import CCDData

from scipy import optimize

%matplotlib inline
import matplotlib.pyplot as plt


# Read image

In [ ]:
ccd = CCDData.read('kelt-1-055R.fit', unit='adu')   #Read in a single fits image

In [ ]:
# This function really should be somewhere else eventually.
def scale_and_downsample(data, downsample=4,
                         min_percent=20,
                         max_percent=99.5):

    scaled_data = scale_image(data,
                              min_percent=min_percent,
                              max_percent=max_percent)

    if downsample > 1:
        scaled_data = block_reduce(scaled_data,
                                   block_size=(downsample, downsample))
    return scaled_data


In [ ]:
#use the catalog_search function to find the apass stars in the frame of the image read above
apass, apass_x, apass_y = df.catalog_search(ccd.wcs, ccd.shape, 'II/336/apass9', 
                                            ra_column='RAJ2000', 
                                            dec_column='DEJ2000',
                                            radius=2.0, 
                                            clip_by_frame=False)

#Get any known variable stars from a new catalog search of VSX
vsx, vsx_x, vsx_y = df.catalog_search(ccd.wcs, ccd.shape, 'B/vsx/vsx', 
                                      ra_column='RAJ2000', 
                                      dec_column='DEJ2000',
                                      radius=2.0,
                                      clip_by_frame=False)
vsx_names = vsx['Name']      #Get the names of the variables

In [ ]:
#Creates a boolean array of the apass stars that have well defined magnitudes and color
keep_rows_that_have = {
    'e_r_mag': '<0.05',
    'u_e_r_mag': '=0',
    'e_B-V': '<0.1'
}

apass_in_bright2 = df.catalog_clean(apass, remove_rows_with_mask=True, **keep_rows_that_have)

apass_x, apass_y = ccd.wcs.all_world2pix(apass_in_bright2['RAJ2000'], apass_in_bright2['DEJ2000'], 0)
#create new lists of apass stars and x y pixel coordinates using boolean array
#apass_in_bright, in_apass_x, in_apass_y = apass[apass_bright], apass_x[apass_bright], apass_y[apass_bright]

In [ ]:
disp = scale_and_downsample(ccd.data, downsample=1)

In [ ]:
plt.figure(figsize=(12, 7))
plt.imshow(disp, cmap='gray', origin='lower')
plt.scatter(vsx_x, vsx_y, c='none', s=100, edgecolor='cyan')
plt.title('Blue: VSX, Yellow: APASS', fontsize=20)

for x, y, m in zip(vsx_x, vsx_y, vsx_names):
    plt.text(x, y, str(m), fontsize=18, color='cyan')
    
plt.scatter(apass_x, apass_y, c='none', s=50, edgecolor='yellow', alpha=0.5, marker='o')

for x, y, c in zip(apass_x, apass_y, apass_in_bright2):
    plt.text(x, y, c['r_mag'], fontsize=12, color='yellow')

plt.xlim(0, ccd.shape[1])
plt.ylim(0, ccd.shape[0])

In [ ]:
#Read in the raw measurements file in case the parse function doesn't work as expected
aij_raw = Table.read('all_kelt_1_good_rows.csv')
#Get the sources
#sources = uniformize_source_names(aij_raw)

#Use glowing waffles to parse the measurements file
aij_stars = parse_aij_table('all_kelt_1_good_rows.csv')

In [ ]:
#get the ra (which aij gives in hour angle) from the raw aij data
aij_ra = [np.mean(aij_stars[source].ra) for source in range(len(aij_stars))]
#get the dec from the raw aij data
aij_dec = [np.mean(aij_stars[source].dec) for source in range(len(aij_stars))]
#get the julian date from parsed measurements file
aij_jd = aij_stars[0].jd_utc_start


#Create a list of aij coordinates using apycoord.SkyCoord function
aij_coordinates = apycoord.SkyCoord(aij_ra, aij_dec, unit=(u.deg, u.deg))
#create a list of well defined apass stars' coordinates using apycoord.SkyCoord function
apass_coordinates = apycoord.SkyCoord(apass_in_bright2['RAJ2000'], 
                                      apass_in_bright2['DEJ2000'], 
                                      unit='deg')

vsx_coordinates = apycoord.SkyCoord(vsx['RAJ2000'], vsx['DEJ2000'], unit='deg')




In [ ]:
vsx_idx, apass_idx, d2d, d3d = apass_coordinates.search_around_sky(vsx_coordinates, 1 * u.arcsec)
if len(set(apass_idx)) != len(apass_idx):
    raise RuntimeError('Multiple matches found.')

print(len(apass_in_bright2))
# Remove any APASS stars that are in VSX
print(apass_idx)
print(vsx_coordinates[vsx_idx])
print(apass_coordinates[apass_idx])
if len(apass_idx):
    apass_in_bright2.remove_rows(apass_idx)
    
print(len(apass_in_bright2))
# Recreate APASS coordinate list after deleting some entries
apass_coordinates = apycoord.SkyCoord(apass_in_bright2['RAJ2000'], 
                                      apass_in_bright2['DEJ2000'], 
                                      unit='deg')

In [ ]:
plt.plot(aij_stars[4].magnitude)     #Plot one of the stars just to see what it looks like
plt.ylabel('Instrumental Mag')
plt.xlabel('Image')

#compare different ways to get magnitudes.
#print(aij_mags[0][0], aij_stars[0].magnitude[0])
#print(-2.5*np.log10(174940.8)+2.5*np.log10(60))

# APASS Filter Corrections
## Transform the APASS r magnitudes into R magnitudes using APASS r and i magnitudes

The equation used is R-feder - r-apass = A*c**3 + B*c**2 + C*c + D

Where...

In [ ]:
#Transform the apass magnitudes into the R filter we use
apass_R_mags = df.filter_transform(apass_in_bright2, 'R', r='r_mag', i='i_mag')

In [ ]:

apass_in_bright2.add_column(Column(data=apass_R_mags, name='R'))

In [ ]:
# Yuck. Just....yuck

def single_time_table(stars, row):
    mags = []
    mag_err = []
    RA = []
    Dec = []
    
    for star in stars:
        mags.append(star.magnitude[row])
        RA.append(star.ra[row].value)
        Dec.append(star.dec[row])
        SNR = star.counts[row]/star.error[row]
        mag_err.append(1.0857/SNR)
    mags = Column(name='R', data=mags)
    RA = Column(name='RA', data=RA, unit='degree')
    Dec = Column(name='Dec', data=Dec, unit='degree')
    errs = Column(name='e_R', data=mag_err)
    
    return Table([mags, RA, Dec, errs])

In [ ]:
aij_index, apass_index, d2d, d3d = apass_coordinates.search_around_sky(aij_coordinates, 1 * u.arcsec)

# Check for bad match (more than one object in APASS matches one object in AIJ)
# 

# This checks for duplicates, because a python set consists of the unique elements
# of a list.
assert len(set(aij_index)) == len(aij_index)

In [ ]:
apass_color = apass_in_bright2['Bmag'][apass_index] - apass_in_bright2['Vmag'][apass_index]
aij_table = single_time_table(aij_stars, 0)[aij_index]
transform_params = df.calculate_transform_coefficients(aij_table['R'], 
                                                       apass_in_bright2['R'][apass_index], 
                                                       apass_color,
                                                       input_mag_error=aij_table['e_R'])

In [ ]:
transform_params

In [ ]:
plt.plot(apass_color, apass_in_bright2['R'][apass_index] - aij_table['R'], 'o')
xlims = np.array(plt.xlim())
plt.plot(xlims, transform_params[1] + transform_params[0] * xlims)

In [ ]:
#Create empty list for the corrections (slope and intercept)
corrections = []
#Create empy list for the error in the corrections
fit_error = []
all_Rminusr_error = []
#loop over all images
for idx in range(len(aij_stars[0].ra)):
    # Calculate APASS color
    #    NOTE: why not use the B-V column???
    apass_color = apass_in_bright2['Bmag'][apass_index] - apass_in_bright2['Vmag'][apass_index]
    
    aij_table = single_time_table(aij_stars, idx)[aij_index]
    transform_params = df.calculate_transform_coefficients(aij_table['R'], 
                                                           apass_in_bright2['R'][apass_index], 
                                                           apass_color,
                                                           input_mag_error=aij_table['e_R'])
    corrections.append(transform_params)
    # Plot every 25 fits...
    if not (idx % 25):
        #Plot the BminusV vs Rminusr graph for the first image
        Rminusr = apass_in_bright2['R'][apass_index]-aij_table['R']
        plt.plot(apass_color, Rminusr, 'o')
        x = np.array(apass_color)
        plt.plot(x, Rminusr, '+')
        plt.plot(x, transform_params[0] * x + transform_params[1])
        plt.title(str(idx))
        plt.show()



In [ ]:
plt.figure(figsize=(10, 5))
#This just plots the slope of the linear fit for all images
for image in range(len(aij_stars[0].ra)):
    time = Time(aij_stars[0].jd_utc_start[image], scale='utc', format='jd')
    ax = plt.scatter(image, corrections[image][0])
plt.ylabel('Color correction')
plt.xlabel('Image number')

In [ ]:
for image in range(len(aij_stars[0].ra)):
    plt.scatter(aij_stars[0].airmass[image], corrections[image][0])
plt.ylabel('Color correction')
plt.xlabel('airmass')
plt.xlim(1, 2)

In [ ]:
plt.figure(figsize=(10, 5))
#This just plots the slope of the linear fit for all images
for image in range(len(aij_stars[0].ra)):
    time = Time(aij_stars[0].jd_utc_start[image], scale='utc', format='jd')
    ax = plt.scatter(image, corrections[image][1])
plt.ylabel('Zero point')
plt.xlabel('Image number')

• Source Radius: radius of the aperture used to calculate
net integrated counts. In fixed aperture mode, this value
is the aperture radius set by the user. In variable aperture
mode, this value represents the actual aperture radius
calculated as the product of the average FWHM in
the image and the multiplicative factor set in the MultiAperture
Measurements set up panel.

• FWHM mult: in variable aperture mode, this value is
the FWHM multiplier set in the Multi-Aperture Measurements
set up panel. In fixed aperture mode, this
column is not included in the table.

• Source Rad(base): in variable aperture mode, this
value represents the fixed aperture radius set by the user
and should be set to a number greater than 1.5 times the
maximum FWHM expected to ensure proper measurement
of FWHM. In fixed aperture mode, this column is
not included in the table.

• Sky Rad(min): radius of the inner edge of the annulus
used to calculate the sky background

• Sky Rad(max): radius of the outer edge of the annulus
used to calculate the sky background

In [ ]:
def source_error(source_number):             #Returns the column heading for a star's source-sky error in the measurements file
    col_name = 'Source-Sky_C'+str(source_number + 1)
    return col_name

#Calculate the error in the magnitudes to be used in the apass calibration
#%matplotlib inline
#define the gain of the ccd
gain = 1.5
#define the read noise of the ccd
read_noise = 30.0


npix = np.pi * 15**2  # pixel^2, estimated aperture size
n_sky = 50.0   # counts/pixel, estimated upper limit

#create a magnitude error list
mag_err = []
for source in range(len(aij_stars)):
    #use source_error function to get error in source-sky
    err = aij_stars[source].error
    #calculate the signal to noise ratio
    #     gain * aij_raw[source_column(source)]
    snr = aij_stars[source].counts / err
    #calculate magnitude error
    mag_e = 1/snr
    #add magnitude error to list mag_err
    mag_err.append(mag_e)

In [ ]:
mag_err[0]

Make a table with columns for: 

+ time
+ instrumental magnitudes
+ transform color term
+ transform zero point 
+ APASS color

Then the transform for each star is just a matter of multiplying/adding columns.

Well, or maybe not.

In [ ]:
apass_coordinates = apycoord.SkyCoord(apass['RAJ2000'], 
                                      apass['DEJ2000'], 
                                      unit='deg')
all_apass_index, d2d, d3d = aij_coordinates.match_to_catalog_sky(apass_coordinates)

good = d2d < 3 * u.arcsec
if not good.all():
    print("Darn.")
    print(good.sum())
    print(d2d)
    
apass_colors = apass['B-V'][all_apass_index]
apass_color_errors = apass['e_B-V'][all_apass_index]

In [ ]:
#create an array to store the corrected curves
corrected_curves = [] # np.zeros_like(aij_stars[0].magnitude)
#create an array to store the error in the corrected curves
corrected_curves_er = [] # np.zeros_like(corrected_curves)
#create an array to store the signal to noise ratios for all stars
all_SNR = [] # np.zeros_like(corrected_curves)
#loop over all of the stars

corrections = np.array(corrections)

for obj, star in enumerate(aij_stars):
    #print a little update just to know where the progress is at
    print('here in the object loop, we are now studying object number', str(obj))

    #get the error in the apass color for the star
    er_BV = apass_color_errors[obj]
    #get the apass color for the star
    BminusV = apass_colors[obj]
    #loop over all of the images
    
    transformed_mag = star.magnitude + apass_colors[obj] * corrections[:, 0] + corrections[:, 1]
    corrected_curves.append(np.array(transformed_mag))

#Plot the corrected light curve of star index 0
plt.plot(corrected_curves[5])    #looks kinda screwed up... shouldn't they meet into eachother smoothly with the 
                                 #apass corrections. They are on different nights and all but still...?

In [ ]:
#create an array of the dates each image was taken at (JD rounded to day)
night = np.floor(np.array(aij_stars[0].mjd_start + 0.5)) -1
#print(night)
#find the unique nights in the array of nights
unique_nights = np.unique(night)
#take out one of the nights if you want
unique_nights = set(unique_nights)# - set([57249.0])
#sort the list of nights obviously
unique_nights = sorted(unique_nights)
#find the number of nights to be used for plotting
number_of_nights = len(unique_nights)

night_stdev = [[] for a_night in range(number_of_nights)]
for index, star in enumerate(aij_stars):
    
    #loop over all of the nights and their index in unique nights
    for i, this_night in enumerate(unique_nights):
        #create a night mask that is a boolean in the shape of night
        night_mask = (night == this_night)
        stdev = np.std(corrected_curves[index][night_mask])
        night_stdev[i].append(stdev)

comp_stars = []

for index, stdevs in enumerate(night_stdev):
    good_comps = list(np.argpartition(np.array(stdevs),5)[2:7])
    comp_stars.append(good_comps)

## Functions for making single magnitude plot
### This section is taken from the multi-night photometry notebook from a kelt-1 notebook.

In [ ]:
#Define the function to be used to plot the differential magnitudes
def plot_magnitudes(mags=None, errors=None, times=None, source=None, night=None, ref_mag=0, color=None, name=''):
    #calcualte the mean of the magnitudes passed
    mean = mags.mean()
    #calcualte the standard deviation of the magntiudes passed
    std = mags.std()
    #plot the magnitudes vs time
    plt.errorbar(times, mags, yerr=errors, fmt='o',
                 label='{}, stdev: {:5.3f}\nnight: {}'.format(source, std, night))
    #change the xlims of the plot to reflect the times
    plt.xlim(times.min(), times.max())
    #Plots a line correspinding to the mean
    plt.plot(plt.xlim(), [mean, mean], 'k--', )
    #plots a line corresponding to the upper limit of the mean
    plt.plot(plt.xlim(), [mean + std, mean + std], 'k:')
    #plots a line corresponding to the lower limit of the mean
    plt.plot(plt.xlim(), [mean - std, mean - std], 'k:')
    #plt.plot(pd.rolling_mean(times, 20, center=True), 
    #         pd.rolling_mean(mags, 20, center=True),
    #         color='gray', linewidth=3)
    # Make sure plot range is at least 0.1 mag...
    min_range = 0.1
    #find the ylim of the plot
    ylim = plt.ylim()
    #check if the difference in the limits is less then the min range
    if ylim[1] - ylim[0] < min_range:
        #if less then the mid range then change the y limits to be min_range different
        plt.ylim(mean - min_range/2, mean + min_range/2)
    
    #find the new ylim of the plot
    ylim = plt.ylim()
    # Reverse vertical axis so brighter is higher
    plt.ylim((ylim[1], ylim[0]))
    plt.title(str(color) + name)
    plt.legend()
    #send back the mean and the standard deviation of the plot
    return mean, std

In [ ]:
#define array for the comparison magnitudes
corrected_counts = 10**(-(corrected_curves-2.5*np.log10(aij_raw['EXPOSURE']))/2.5)
comp_counts = []

for i, this_night in enumerate(unique_nights):
        #create a night mask that is a boolean in the shape of night
        night_mask = (night == this_night)
        comp = np.zeros(sum(night_mask))
        for star in comp_stars[i]:
            if star not in (4, 12, 17):
                comp += corrected_counts[star][night_mask]
        comp_counts += list(comp)
comp_counts = np.array(comp_counts)

#calculate the differential magnitudes by simply taking the difference between the corrected curves and the comparison magnitudes"""
diff_corrected = -2.5*np.log10(corrected_counts/comp_counts)

In [ ]:
vsx_index, d2d, d3d = aij_coordinates.match_to_catalog_sky(vsx_coordinates)

vsx_match = d2d < 1 * u.arcsec

In [ ]:
#loop over all of the stars and their index
for index, star in enumerate(aij_stars):
    #start a figure...
    plt.figure(figsize=(5*number_of_nights, 5))
    
    #define a list for night means
    night_means = []
    #define a list for night standard deviations
    night_stds = []
    #define a list for night bins? I suppose
    night_bins = []
    #get the color of the star
    BminusV = apass_colors[index]
    if vsx_match[index]:
        name = " VAR: " + vsx_names[vsx_index[index]]
    else:
        name = ''
    #loop over all of the nights and their index in unique nights
    for i, this_night in enumerate(unique_nights):
        plt.subplot(1, number_of_nights + 1, i + 1)
        #create a night mask that is a boolean in the shape of night
        night_mask = (night == this_night)
        #get the night mean and std from the plot magnitudes function and plot the magnitudes
        night_mean, night_std = plot_magnitudes(mags=corrected_curves[index][night_mask], 
                                                times=aij_raw['BJD_TDB'][night_mask], source = index+1,
                                               night = this_night, color = BminusV, name=name)
""", errors = corrected_curves_er[index][night_mask]"""